In [16]:
from k8sclient.keywords import (
    list_ready_nodes,
    get_pod_ip,
    wait_for_pod_state,
    RUNNING,
    SUCCEEDED,
    tail_pod_logs,
    delete_pod,
    # delete_service,
    cleanup_pods,
    cleanup_services,
    NOT_FOUND
)
from k8sclient.Components import PodBuilder, ServicePort, ServiceBuilder
import re
import time

In [2]:
image = "127.0.0.1:30100/library/alpine-iperf"
server_args = "iperf -f M -i 1 -m -s"
client_args = r"iperf -f M -t 10 -i 1 -c %s"
namespace = "k8sft"
global_server_name = "iperf-server"

In [3]:
server_port = ServicePort("serverport", 5001, 5001)
server_service = ServiceBuilder(global_server_name, namespace).add_port(server_port)

In [12]:
# deploy servers
def deploy_server(node):
    node_name = "-".join(node.split("."))
    service_name = "server-" + node_name
    pod_name = "server-pod-" + node_name
    service = ServiceBuilder(service_name, namespace).add_port(server_port)
    PodBuilder(
        pod_name, namespace
    ).add_container(
        name=pod_name + "-container",
        image=image,
        args=server_args,
        limits={'cpu': '2', 'memory': '4Gi'},
        requests={'cpu': '0', 'memory': '0'},
        ports=[server_port]
    ).set_node(node).attache_service(service).deploy()
    wait_for_pod_state(namespace, pod_name, timeout=600, expect_status=RUNNING)
    service.deploy(force=True)

def undeploy_server(node):
    node_name = "-".join(node.split("."))
    service_name = "server-" + node_name
    pod_name = "server-pod-" + node_name
    delete_pod(namespace, pod_name)
    delete_service(namespace, service_name)
    
# start client test. servers is a list of (pod_ip, node)
def deploy_client(node, servers):
    node_name = "-".join(node.split("."))
    pod_name = "client-pod-" + node_name
    r = {}
    for n in servers:
        server_node_name = "-".join(n.split("."))
        server_pod_name = "server-pod-" + server_node_name
        pod_ip = get_pod_ip(namespace, server_pod_name)
        PodBuilder(
            pod_name, namespace
        ).add_container(
            name=pod_name + "-container",
            image=image,
            args=client_args % pod_ip,
            limits={'cpu': '1', 'memory': '4Gi'},
            requests={'cpu': '0', 'memory': '0'},
        ).set_node(node).deploy()
        wait_for_pod_state(namespace, pod_name, timeout=600, expect_status=SUCCEEDED)
        logs = tail_pod_logs(namespace, pod_name, lines=20).strip()
        summary = logs.split("\n")[-1]
        m = re.match(r".*[^.\d]+([.\d]+)\s+MBytes/sec", summary)
        r[n] = float(m.group(1))
        print "%s -> %s : %f MBytes/sec" % (node, n, r[n])
        delete_pod(namespace, pod_name)
        wait_for_pod_state(namespace, pod_name, timeout=240, expect_status=NOT_FOUND)
    return r

In [31]:
# start the test
# nodes = list_ready_nodes()
# [deploy_server(_) for _ in nodes]
#reports = [deploy_client(_, nodes) for _ in nodes]
#[undeploy_server(_) for _ in nodes]
    

In [17]:
# cleanup
# cleanup_pods(namespace)
# cleanup_services(namespace)

In [ ]:
# peer to peer test


def run_client(client_node, server_node, server_pod_ip):
    client_pod_name = "client-" + "-".join(client_node.split("."))
    PodBuilder(
        client_pod_name, namespace
    ).add_container(
        name=client_pod_name + "-container",
        image=image,
        args=client_args % server_pod_ip,
        limits={'cpu': '0.5', 'memory': '512Mi'},
        requests={'cpu': '0', 'memory': '0'},
    ).set_node(client_node).deploy()
    # wait for client complete
    wait_for_pod_state(namespace, client_pod_name, timeout=600, expect_status=SUCCEEDED)
    # parse client log to get tx speed.
    logs = tail_pod_logs(namespace, client_pod_name, lines=20).strip()
    # print logs
    summary = logs.split("\n")[-1]
    m = re.match(r".*[^.\d]+([.\d]+ MBytes/sec)", summary)
    print server_node, client_node, server_pod_ip, m.group(1)
    delete_pod(namespace, client_pod_name)
    wait_for_pod_state(namespace, client_pod_name, timeout=240, expect_status=NOT_FOUND)
    
    
def test(server_node, client_node):
    print client_node, "->", server_node
    server_pod_name = "server-" + "-".join(server_node.split("."))
    PodBuilder(
        server_pod_name, namespace
    ).add_container(
        name=server_pod_name + "-container",
        image=image,
        args=server_args,
        limits={'cpu': '1', 'memory': '4Gi'},
        requests={'cpu': '0', 'memory': '0'},
        ports=[server_port]
    ).set_node(server_node).attache_service(server_service).deploy()
    # wait for server pod running
    wait_for_pod_state(namespace, server_pod_name, timeout=600, expect_status=RUNNING)
    time.sleep(3)
    # get server pod ip
    server_pod_ip = get_pod_ip(namespace, server_pod_name)
    run_client(client_node, server_node, server_pod_ip)
    run_client(client_node, server_node, global_server_name)
    delete_pod(namespace, server_pod_name)
    wait_for_pod_state(namespace, server_pod_name, timeout=240, expect_status=NOT_FOUND)

In [ ]:
server_service.deploy(force=True)
test("10.19.137.140", "10.19.137.142")
server_service.un_deploy()

In [37]:
import pandas as pd
df = pd.DataFrame(reports, index=nodes)
df.columns = df.columns.str.split(".").str.get(-1)

In [54]:
# pod to pod iperf report in MBytes/sec
df.loc["10.19.137.140"].reset_index()

,index,10.19.137.140
0,140,5535.0
1,141,997.0
2,142,1018.0
3,143,1001.0
4,144,993.0
5,145,996.0
6,146,1025.0
7,147,112.0
8,148,112.0
9,149,112.0
